# NLP Project
Nelson VICEL--FARAH\
Karen KASPAR\
Romain BRAND\

## The datasets

In [1]:
from datasets import load_dataset

In [2]:
imdb_dataset = load_dataset("imdb")

Reusing dataset imdb (/home/miolith/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

#### 1. On observe que le dataset IMDb a 3 splits différents
#### 2. Le **train** split a 25 000 lignes, **test** 25 000 lignes et **unsupervised** avec 50 000 lignes

In [9]:
import pandas as pd

train_df = pd.DataFrame(imdb_dataset["train"])

train_df["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [10]:
test_df = pd.DataFrame(imdb_dataset["test"])

test_df["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

#### 3. Il y a 50% de chaque classe. Le dataset est équilibré.

## Naive Bayes classifier

### 1. Preprocessing

In [16]:
from string import punctuation

# i. Lower case the text

train_df["text"] = train_df["text"].str.lower()
test_df["text"] = test_df["text"].str.lower()

# ii. Remove punctuation

train_df["text"] = train_df["text"].str.replace('[{}]'.format(punctuation), ' ')
test_df["text"] = test_df["text"].str.replace('[{}]'.format(punctuation), ' ')

train_df.head()

/tmp/ipykernel_48782/2959889837.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df["text"] = train_df["text"].str.replace('[{}]'.format(punctuation), '')
/tmp/ipykernel_48782/2959889837.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df["text"] = test_df["text"].str.replace('[{}]'.format(punctuation), '')


,text,label
0,i rented i am curiousyellow from my video stor...,0
1,i am curious yellow is a risible and pretentio...,0
2,if only to avoid making this type of film in t...,0
3,this film was probably inspired by godards mas...,0
4,oh brotherafter hearing about this ridiculous ...,0


### 2. Implémenter Naive Bayes

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(CountVectorizer(), MultinomialNB())

pipe.fit(train_df["text"], train_df["label"])

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

### 3. Précision

In [24]:
pipe.score(train_df["text"], train_df["label"])

0.91284

In [25]:
pipe.score(test_df["text"], test_df["label"])

0.81728

### 4. La précision est suffisante dans notre cas car le dataset est équilibré, donc on peut facilement juger qu'une précision supérieure à 50% est meilleure qu'un algorithme aléatoire.

### 6.

In [27]:
test_df["prediction"] = pipe.predict(test_df["text"])
test_df["prediction"]

0        0
1        0
2        0
3        0
4        0
        ..
24995    0
24996    0
24997    1
24998    0
24999    0
Name: prediction, Length: 25000, dtype: int64

In [42]:
failed_sample1 = test_df[(test_df["prediction"] == 1) & (test_df["label"] == 0)].iloc[1,:]["text"]
failed_sample2 = test_df[(test_df["prediction"] == 0) & (test_df["label"] == 1)].iloc[1,:]["text"]

Cet exemple est un faux positif

In [43]:
failed_sample1

'ben rupert grint is a deeply unhappy adolescent the son of his unhappily married parents his father nicholas farrell is a vicar and his mother laura linney is  well lets just say shes a somewhat hypocritical soldier in jesus army its only when he takes a summer job as an assistant to a foulmouthed eccentric oncefamous and nowforgotten actress evie walton julie walters that he finally finds himself in true harold and maude fashion of course evie is deeply unhappy herself and its only when these two sad sacks find each other that they can put their mutual misery aside and hit the road to happinessbr br of course its corny and sentimental and very predictable but it has a hard side to it too and walters who could sleepwalk her way through this sort of thing if she wanted is excellent its when she puts the craziness to one side and finds the pathos in the character like hitting the bottle and throwing up in the sink that shes at her best the problem is shes the only interesting character 

Cet exemple est un faux négatif

In [44]:
failed_sample2

'this was a bold movie to hit indian cinemas when it was released the first movie to perhaps openly depict lesbian tendencies amongst indian women the leading actress of indian cinema shabana azmi added substance to the movie with her hot passionate scenes with nandita dasbr br the movie oozed with sexuality and the director used sex in the best way possible the sex was not for erotic purposes but was in the context of the movie the scene where nandita das loses her virginity to her husband certainly was the first of its kind in indian cinemasbr br good acting by all the actors especially nandita das amidst criticism from the indian public'